In [1]:
print(1)

1


In [2]:
!pip list


Package            Version
------------------ ------------
asttokens          3.0.0
colorama           0.4.6
comm               0.2.3
debugpy            1.8.16
decorator          5.2.1
exceptiongroup     1.3.0
executing          2.2.0
importlib_metadata 8.7.0
ipykernel          6.30.1
ipython            8.18.1
jedi               0.19.2
jupyter_client     8.6.3
jupyter_core       5.8.1
matplotlib-inline  0.1.7
nest_asyncio       1.6.0
numpy              2.0.2
packaging          25.0
parso              0.8.4
pickleshare        0.7.5
pillow             11.3.0
pip                25.3
platformdirs       4.3.8
prompt_toolkit     3.0.51
psutil             7.0.0
pure_eval          0.2.3
Pygments           2.19.2
python-dateutil    2.9.0.post0
pywin32            311
pyzmq              27.0.2
setuptools         80.9.0
six                1.17.0
stack_data         0.6.3
torch              1.9.1+cu111
torchaudio         0.9.1
torchvision        0.10.1+cu111
tornado            6.5.2
traitlets       

In [ ]:
[Interface]
PrivateKey = # ← 这是自己的私钥
Address = 10.0.0.6/24
DNS = 4.4.4.4


[Peer]
PublicKey  =   # ← 这是服务器的公钥
AllowedIPs = 0.0.0.0/0, ::/0
Endpoint = 8.138.164.133:51820
PersistentKeepalive = 25


In [ ]:
sudo wg-quick down wg0 || true
sudo wg-quick up wg0
sudo wg

In [ ]:
cd /etc/wireguard
sudo nano wg0.conf

一、症状

客户端把 AllowedIPs 改为 0.0.0.0/0 后：启用隧道立刻“什么都连不上”

但之前 AllowedIPs = 10.0.0.0/24（分流）时可正常握手、keepalive 正常

二、服务器端核查（确认不是服务器 NAT/转发问题）
1）确认开启转发
sudo sysctl net.ipv4.ip_forward


期望输出：

net.ipv4.ip_forward = 1

2）确认默认出网网卡
ip route | grep default


你机器输出：

default ... dev eth0 ...


结论：出网网卡就是 eth0。

3）确认 NAT 生效（MASQUERADE）
sudo iptables -t nat -L POSTROUTING -n -v


你机器输出里有：

MASQUERADE ... out eth0 ...


结论：NAT 规则存在且出口正确。

4）确认 FORWARD 有流量（说明隧道有转发能力）
sudo iptables -L FORWARD -n -v


你机器显示计数很大（数百万包），结论：转发链路可工作。

✅ 所以：服务器端基础网络（转发 + NAT + 出口网卡）是 OK 的。

四、修复方法（推荐方案：Windows 添加 Endpoint 静态路由）
Step 1：保持客户端全局 AllowedIPs

客户端配置（关键部分）：

AllowedIPs = 0.0.0.0/0, ::/0
Endpoint = 8.138.164.133:51820
PersistentKeepalive = 25

Step 2：在 Windows 添加“到服务器公网 IP 的例外路由”

先查你当前本地默认网关（未开 VPN 时）
打开 CMD：

ipconfig


找到你正在用的网卡，记下 “默认网关”，例如 192.168.1.1

以管理员运行 CMD，添加永久路由：

route -p add 8.138.164.133 mask 255.255.255.255 192.168.1.1


把 192.168.1.1 换成你真实默认网关

这条路由的意思：去服务器公网 IP 永远走本地网关，不走 VPN

重新启用 WireGuard 隧道测试

Step 3：验证

ping 10.0.0.1（通：隧道通）

ping 8.8.8.8（通：全局出网通）

打开网页查公网 IP（应变成服务器出口 IP）

如果能 ping IP 但域名不行，把 DNS 换成 1.1.1.1 或 8.8.8.8 再试。